In [ ]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});